# import neccessary libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# read csv file

In [2]:
df = pd.read_csv("Train(1).csv")
df.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,dcf68cc2fb515ccad7d8b9b3bd80ee2a4b270063,SAINT-LOUIS,K > 24 month,17000.0,32.0,18000.0,6000.0,34.0,NaN,97.0,355.0,6.0,NaN,NaN,NO,62,All-net 500F=2000F;5d,35.0,0
1,71c44b5ba328db5c4192a80f7cf8f244d9350ed0,NaN,K > 24 month,4300.0,29.0,4427.0,1476.0,37.0,1764.0,8.0,3.0,0.0,NaN,2.0,NO,40,"Data: 100 F=40MB,24H",22.0,0
2,ce46411b1526c94f20a383b8cb188f8d27f82a0a,TAMBACOUNDA,K > 24 month,1500.0,3.0,1500.0,500.0,3.0,NaN,30.0,30.0,NaN,NaN,NaN,NO,32,All-net 500F=2000F;5d,3.0,0
3,f467cdb6669818373c26c2bad44e01ba66f97d21,FATICK,K > 24 month,1500.0,3.0,2497.0,832.0,4.0,0.0,159.0,45.0,19.0,NaN,NaN,NO,18,On net 200F=Unlimited _call24H,3.0,0
4,ec45e1a1888a32b5dcce0954cfec20c6e037db31,FATICK,K > 24 month,NaN,NaN,498.0,166.0,3.0,1.0,1.0,3.0,NaN,NaN,NaN,NO,50,NaN,NaN,0


In [3]:
df.info()
# there are a lot of null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 19 columns):
user_id           400000 non-null object
REGION            242480 non-null object
TENURE            400000 non-null object
MONTANT           259723 non-null float64
FREQUENCE_RECH    259723 non-null float64
REVENUE           265337 non-null float64
ARPU_SEGMENT      265337 non-null float64
FREQUENCE         265337 non-null float64
DATA_VOLUME       203146 non-null float64
ON_NET            254181 non-null float64
ORANGE            233683 non-null float64
TIGO              160614 non-null float64
ZONE1             31690 non-null float64
ZONE2             25513 non-null float64
MRG               400000 non-null object
REGULARITY        400000 non-null int64
TOP_PACK          232671 non-null object
FREQ_TOP_PACK     232671 non-null float64
CHURN             400000 non-null int64
dtypes: float64(12), int64(2), object(5)
memory usage: 58.0+ MB


In [4]:
#drop columns
df = df.drop(["user_id","TOP_PACK","MRG","TENURE","MONTANT","ARPU_SEGMENT","FREQUENCE_RECH"], axis = "columns")

In [5]:
df.head()

,REGION,REVENUE,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK,CHURN
0,SAINT-LOUIS,18000.0,34.0,NaN,97.0,355.0,6.0,NaN,NaN,62,35.0,0
1,NaN,4427.0,37.0,1764.0,8.0,3.0,0.0,NaN,2.0,40,22.0,0
2,TAMBACOUNDA,1500.0,3.0,NaN,30.0,30.0,NaN,NaN,NaN,32,3.0,0
3,FATICK,2497.0,4.0,0.0,159.0,45.0,19.0,NaN,NaN,18,3.0,0
4,FATICK,498.0,3.0,1.0,1.0,3.0,NaN,NaN,NaN,50,NaN,0


# Replace categorical columns with the mode and the numerical columns with the mean using simple imputer

In [6]:
#region
from sklearn.impute import SimpleImputer
region_impute = SimpleImputer(missing_values=np.nan,strategy="most_frequent")
df[["REGION"]] = region_impute.fit_transform(df[["REGION"]]) #fills with the mode

In [7]:
#FREQUENCY top pack
freq_top_pack_impute = SimpleImputer(missing_values=np.nan,strategy="mean")
df[["FREQ_TOP_PACK"]] = freq_top_pack_impute.fit_transform(df[["FREQ_TOP_PACK"]]) #fills with the mode

In [8]:
#remaining numerical columns
numerical_impute = SimpleImputer(missing_values=np.nan,strategy="mean")
df.iloc[:,1:9] = numerical_impute.fit_transform(df.iloc[:,1:9]) 

In [9]:
df.info()#all null values have been replaced.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 12 columns):
REGION           400000 non-null object
REVENUE          400000 non-null float64
FREQUENCE        400000 non-null float64
DATA_VOLUME      400000 non-null float64
ON_NET           400000 non-null float64
ORANGE           400000 non-null float64
TIGO             400000 non-null float64
ZONE1            400000 non-null float64
ZONE2            400000 non-null float64
REGULARITY       400000 non-null int64
FREQ_TOP_PACK    400000 non-null float64
CHURN            400000 non-null int64
dtypes: float64(9), int64(2), object(1)
memory usage: 36.6+ MB


In [10]:
#drop the target variable
X = df.drop(["CHURN"], axis="columns")

In [11]:
#target variable
y = df["CHURN"]

In [12]:
#encode the categorical variables using column transformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [13]:
#region
region = ColumnTransformer(transformers=[("encoder",OneHotEncoder(),[0])],remainder="passthrough")
X = pd.DataFrame(region.fit_transform(X),columns = region.get_feature_names())

In [14]:
X.head()

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


,encoder__x0_DAKAR,encoder__x0_DIOURBEL,encoder__x0_FATICK,encoder__x0_KAFFRINE,encoder__x0_KAOLACK,encoder__x0_KEDOUGOU,encoder__x0_KOLDA,encoder__x0_LOUGA,encoder__x0_MATAM,encoder__x0_SAINT-LOUIS,...,REVENUE,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,18000.0,34.0,3369.763441,97.0,355.0,6.000000,7.874282,7.187003,62.0,35.000000
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4427.0,37.0,1764.000000,8.0,3.0,0.000000,7.874282,2.000000,40.0,22.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1500.0,3.0,3369.763441,30.0,30.0,23.134608,7.874282,7.187003,32.0,3.000000
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2497.0,4.0,0.000000,159.0,45.0,19.000000,7.874282,7.187003,18.0,3.000000
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,498.0,3.0,1.000000,1.0,3.0,23.134608,7.874282,7.187003,50.0,9.254209


In [15]:
region.get_feature_names()

['encoder__x0_DAKAR',
 'encoder__x0_DIOURBEL',
 'encoder__x0_FATICK',
 'encoder__x0_KAFFRINE',
 'encoder__x0_KAOLACK',
 'encoder__x0_KEDOUGOU',
 'encoder__x0_KOLDA',
 'encoder__x0_LOUGA',
 'encoder__x0_MATAM',
 'encoder__x0_SAINT-LOUIS',
 'encoder__x0_SEDHIOU',
 'encoder__x0_TAMBACOUNDA',
 'encoder__x0_THIES',
 'encoder__x0_ZIGUINCHOR',
 'REVENUE',
 'FREQUENCE',
 'DATA_VOLUME',
 'ON_NET',
 'ORANGE',
 'TIGO',
 'ZONE1',
 'ZONE2',
 'REGULARITY',
 'FREQ_TOP_PACK']

In [16]:
Xy = X.iloc[:,:14]
Xy

,encoder__x0_DAKAR,encoder__x0_DIOURBEL,encoder__x0_FATICK,encoder__x0_KAFFRINE,encoder__x0_KAOLACK,encoder__x0_KEDOUGOU,encoder__x0_KOLDA,encoder__x0_LOUGA,encoder__x0_MATAM,encoder__x0_SAINT-LOUIS,encoder__x0_SEDHIOU,encoder__x0_TAMBACOUNDA,encoder__x0_THIES,encoder__x0_ZIGUINCHOR
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
399996,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
399997,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
399998,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# now all features are numerical, seperate the newly encoded categorical variables from the numerical variables for scaling

In [17]:
Xx = X.iloc[:,14:]
Xx

,REVENUE,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK
0,18000.000000,34.000000,3369.763441,97.000000,355.000000,6.000000,7.874282,7.187003,62.0,35.000000
1,4427.000000,37.000000,1764.000000,8.000000,3.000000,0.000000,7.874282,2.000000,40.0,22.000000
2,1500.000000,3.000000,3369.763441,30.000000,30.000000,23.134608,7.874282,7.187003,32.0,3.000000
3,2497.000000,4.000000,0.000000,159.000000,45.000000,19.000000,7.874282,7.187003,18.0,3.000000
4,498.000000,3.000000,1.000000,1.000000,3.000000,23.134608,7.874282,7.187003,50.0,9.254209
...,...,...,...,...,...,...,...,...,...,...
399995,704.000000,3.000000,3369.763441,12.000000,14.000000,23.134608,7.874282,7.187003,46.0,1.000000
399996,5505.487757,13.951835,3369.763441,275.917586,95.532927,23.134608,7.874282,7.187003,14.0,9.254209
399997,699.000000,5.000000,237.000000,1.000000,95.532927,23.134608,7.874282,7.187003,15.0,3.000000
399998,9.000000,1.000000,0.000000,275.917586,0.000000,23.134608,7.874282,7.187003,5.0,9.254209


In [18]:
scaler = MinMaxScaler()
normalised_X2 = pd.DataFrame(scaler.fit_transform(Xx),columns =Xx.columns)                               

In [19]:
normalised_X2.head()

,REVENUE,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK
0,0.077113,0.366667,0.003606,0.002155,0.052298,0.002175,0.004752,0.007109,1.000000,0.054140
1,0.018962,0.400000,0.001887,0.000178,0.000442,0.000000,0.004752,0.001978,0.639344,0.033439
2,0.006422,0.022222,0.003606,0.000667,0.004420,0.008388,0.004752,0.007109,0.508197,0.003185
3,0.010694,0.033333,0.000000,0.003532,0.006629,0.006889,0.004752,0.007109,0.278689,0.003185
4,0.002129,0.022222,0.000001,0.000022,0.000442,0.008388,0.004752,0.007109,0.803279,0.013144


In [20]:
#concatenate boh dataframes
X = pd.concat([Xy,normalised_X2],axis = 1)

In [21]:
X.head()

,encoder__x0_DAKAR,encoder__x0_DIOURBEL,encoder__x0_FATICK,encoder__x0_KAFFRINE,encoder__x0_KAOLACK,encoder__x0_KEDOUGOU,encoder__x0_KOLDA,encoder__x0_LOUGA,encoder__x0_MATAM,encoder__x0_SAINT-LOUIS,...,REVENUE,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.077113,0.366667,0.003606,0.002155,0.052298,0.002175,0.004752,0.007109,1.000000,0.054140
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.018962,0.400000,0.001887,0.000178,0.000442,0.000000,0.004752,0.001978,0.639344,0.033439
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.006422,0.022222,0.003606,0.000667,0.004420,0.008388,0.004752,0.007109,0.508197,0.003185
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.010694,0.033333,0.000000,0.003532,0.006629,0.006889,0.004752,0.007109,0.278689,0.003185
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002129,0.022222,0.000001,0.000022,0.000442,0.008388,0.004752,0.007109,0.803279,0.013144


In [22]:
#split data into training and testing sets
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 42)

In [23]:
X_train.shape

(280000, 24)

In [24]:
X_test.shape

(120000, 24)

In [25]:
y_train.shape

(280000,)

In [26]:
y_test.shape

(120000,)

# using xgboost

In [27]:
from xgboost.sklearn import XGBClassifier  

In [28]:
#create a parameter grid
params = {'colsample_bytree':[0.5,0.6,0.7,0.8,0.9], 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
 'eta': [0.01],
 'eval_metric': ['mae'],
 'max_depth': range(3,10,2),
 'min_child_weight':range(1,6,2),'gamma' : [0,0.1,0.2,0.5,1],'scale_pos_weight':[1],'seed':[27],
 'subsample': [0.5,0.6,0.7,0.8,0.9]
         }

In [29]:
clf =XGBClassifier(learning_rate = 0.1,n_estimators = 1000,objective = "binary:logistic",silent = True,nthread=4)

# using Random forest classifier`

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [31]:
from sklearn.model_selection import StratifiedKFold

folds = 5
param = 5
skf = StratifiedKFold(n_splits = folds,shuffle = True,random_state = 1001)
random_search = RandomizedSearchCV(clf,param_distributions = params,n_iter= param,scoring="roc_auc",n_jobs =4,cv = skf.split(X_train,y_train),verbose = 3,random_state=42)

In [32]:
#fit the train set
random_search.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed: 101.5min finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x000001CCCF864E48>,
                   estimator=XGBClassifier(n_estimators=1000, nthread=4,
                                           silent=True),
                   n_iter=5, n_jobs=4,
                   param_distributions={'colsample_bytree': [0.5, 0.6, 0.7, 0.8,
                                                             0.9],
                                        'eta': [0.01], 'eval_metric': ['mae'],
                                        'gamma': [0, 0.1, 0.2, 0.5, 1],
                                        'max_depth': range(3, 10, 2),
                                        'min_child_weight': range(1, 6, 2),
                                        'reg_alpha': [1e-05, 0.01, 0.1, 1, 100],
                                        'scale_pos_weight': [1], 'seed': [27],
                                        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9]},
                   random_state=42, scoring='roc_auc', verbose=3

In [33]:
random_search.cv_results_

{'mean_fit_time': array([ 531.10358076, 1110.94425015, 1535.48321943,  757.36908045,
         677.56742306]),
 'std_fit_time': array([ 65.61378659,  63.16623993,  89.64062472,  41.73813519,
        166.7965163 ]),
 'mean_score_time': array([ 1.37664518, 11.47531977, 10.054423  ,  2.91290231,  2.69581413]),
 'std_score_time': array([0.2104119 , 3.96390885, 1.70187215, 0.22063916, 0.61394488]),
 'param_subsample': masked_array(data=[0.5, 0.5, 0.5, 0.6, 0.6],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_seed': masked_array(data=[27, 27, 27, 27, 27],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_scale_pos_weight': masked_array(data=[1, 1, 1, 1, 1],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_reg_alpha': masked_array(data=[100, 0.1, 1, 1e-05, 1],
              mask=[False, False, F

In [34]:
 random_search.best_params_

{'subsample': 0.5,
 'seed': 27,
 'scale_pos_weight': 1,
 'reg_alpha': 100,
 'min_child_weight': 5,
 'max_depth': 3,
 'gamma': 1,
 'eval_metric': 'mae',
 'eta': 0.01,
 'colsample_bytree': 0.9}

# predict using the test data set

In [35]:
y_predict = random_search.predict(X_test)
y_predict

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

# check for probabiity of both churn and not churn

In [36]:
y_predict_proba_ = random_search.predict_proba(X_test)

In [37]:
y_predict_proba_

array([[0.9955859 , 0.00441406],
       [0.9955811 , 0.00441893],
       [0.9551925 , 0.04480748],
       ...,
       [0.23454177, 0.7654582 ],
       [0.9919183 , 0.0080817 ],
       [0.7993963 , 0.20060374]], dtype=float32)

# using  log loss 

In [38]:
from sklearn.metrics import log_loss
Log_loss_value = log_loss(y_test,y_predict_proba_)

In [39]:
Log_loss_value

0.2774580152310761

# run all of the above on an unseen data set

In [61]:
test = pd.read_csv("Test(1).csv")
test.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK
0,af900d87e73b7ff6509d2203df4704a98aa5f2a6,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,1,NaN,NaN
1,5335efd940280b82143272275637d1e65d37eadb,NaN,K > 24 month,NaN,NaN,10.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NO,2,NaN,NaN
2,a581f4fa08677c26f83f643248c667e241043086,NaN,K > 24 month,1900.0,15.0,2299.0,766.0,21.0,414.0,NaN,7.0,2.0,NaN,NaN,NO,27,"Data: 100 F=40MB,24H",17.0
3,64f67177d0775262b8087a9e2e3b8061b6324ae6,DAKAR,K > 24 month,3000.0,9.0,2603.0,868.0,14.0,332.0,0.0,23.0,4.0,NaN,NaN,NO,46,IVR Echat_Daily_50F,3.0
4,0d6009a4594c4be22449b8d9cc01a0bcea98faea,DAKAR,K > 24 month,32000.0,47.0,33000.0,11000.0,47.0,NaN,128.0,555.0,280.0,NaN,NaN,NO,61,All-net 500F=2000F;5d,65.0


In [41]:
test = test.drop(["user_id","TOP_PACK","MRG","TENURE","MONTANT","ARPU_SEGMENT","FREQUENCE_RECH"], axis = "columns")
#USE chi square to check the , used tenure 

In [42]:
test.head()

,REGION,REVENUE,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
1,NaN,10.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
2,NaN,2299.0,21.0,414.0,NaN,7.0,2.0,NaN,NaN,27,17.0
3,DAKAR,2603.0,14.0,332.0,0.0,23.0,4.0,NaN,NaN,46,3.0
4,DAKAR,33000.0,47.0,NaN,128.0,555.0,280.0,NaN,NaN,61,65.0


In [43]:
#region
from sklearn.impute import SimpleImputer
region_impute = SimpleImputer(missing_values=np.nan,strategy="most_frequent")
test[["REGION"]] = region_impute.fit_transform(test[["REGION"]]) #fills with the mode

In [44]:
#FREQUENCY top pack
freq_top_pack_impute = SimpleImputer(missing_values=np.nan,strategy="mean")
test[["FREQ_TOP_PACK"]] = freq_top_pack_impute.fit_transform(test[["FREQ_TOP_PACK"]]) #fills with the mode

In [45]:
# rest of the numerical data
numerical_impute = SimpleImputer(missing_values=np.nan,strategy="mean")
test.iloc[:,1:9] = numerical_impute.fit_transform(test.iloc[:,1:9]) 

In [46]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
REGION           100000 non-null object
REVENUE          100000 non-null float64
FREQUENCE        100000 non-null float64
DATA_VOLUME      100000 non-null float64
ON_NET           100000 non-null float64
ORANGE           100000 non-null float64
TIGO             100000 non-null float64
ZONE1            100000 non-null float64
ZONE2            100000 non-null float64
REGULARITY       100000 non-null int64
FREQ_TOP_PACK    100000 non-null float64
dtypes: float64(9), int64(1), object(1)
memory usage: 8.4+ MB


In [47]:
#encoding the categorical variables
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [48]:
region = ColumnTransformer(transformers=[("encoder",OneHotEncoder(),[0])],remainder="passthrough")
X = pd.DataFrame(region.fit_transform(test),columns = region.get_feature_names())

In [49]:
X.head()

,encoder__x0_DAKAR,encoder__x0_DIOURBEL,encoder__x0_FATICK,encoder__x0_KAFFRINE,encoder__x0_KAOLACK,encoder__x0_KEDOUGOU,encoder__x0_KOLDA,encoder__x0_LOUGA,encoder__x0_MATAM,encoder__x0_SAINT-LOUIS,...,REVENUE,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5518.341663,13.979973,3357.428033,279.370703,94.900799,23.459291,8.373307,7.678138,1.0,9.276035
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.000000,1.000000,3357.428033,279.370703,94.900799,23.459291,8.373307,7.678138,2.0,9.276035
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2299.000000,21.000000,414.000000,279.370703,7.000000,2.000000,8.373307,7.678138,27.0,17.000000
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2603.000000,14.000000,332.000000,0.000000,23.000000,4.000000,8.373307,7.678138,46.0,3.000000
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,33000.000000,47.000000,3357.428033,128.000000,555.000000,280.000000,8.373307,7.678138,61.0,65.000000


In [50]:
region.get_feature_names()

['encoder__x0_DAKAR',
 'encoder__x0_DIOURBEL',
 'encoder__x0_FATICK',
 'encoder__x0_KAFFRINE',
 'encoder__x0_KAOLACK',
 'encoder__x0_KEDOUGOU',
 'encoder__x0_KOLDA',
 'encoder__x0_LOUGA',
 'encoder__x0_MATAM',
 'encoder__x0_SAINT-LOUIS',
 'encoder__x0_SEDHIOU',
 'encoder__x0_TAMBACOUNDA',
 'encoder__x0_THIES',
 'encoder__x0_ZIGUINCHOR',
 'REVENUE',
 'FREQUENCE',
 'DATA_VOLUME',
 'ON_NET',
 'ORANGE',
 'TIGO',
 'ZONE1',
 'ZONE2',
 'REGULARITY',
 'FREQ_TOP_PACK']

In [51]:
Xy = X.iloc[:,:14]
Xy

,encoder__x0_DAKAR,encoder__x0_DIOURBEL,encoder__x0_FATICK,encoder__x0_KAFFRINE,encoder__x0_KAOLACK,encoder__x0_KEDOUGOU,encoder__x0_KOLDA,encoder__x0_LOUGA,encoder__x0_MATAM,encoder__x0_SAINT-LOUIS,encoder__x0_SEDHIOU,encoder__x0_TAMBACOUNDA,encoder__x0_THIES,encoder__x0_ZIGUINCHOR
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
99996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
99997,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [52]:
Xx = X.iloc[:,14:]
Xx

,REVENUE,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK
0,5518.341663,13.979973,3357.428033,279.370703,94.900799,23.459291,8.373307,7.678138,1.0,9.276035
1,10.000000,1.000000,3357.428033,279.370703,94.900799,23.459291,8.373307,7.678138,2.0,9.276035
2,2299.000000,21.000000,414.000000,279.370703,7.000000,2.000000,8.373307,7.678138,27.0,17.000000
3,2603.000000,14.000000,332.000000,0.000000,23.000000,4.000000,8.373307,7.678138,46.0,3.000000
4,33000.000000,47.000000,3357.428033,128.000000,555.000000,280.000000,8.373307,7.678138,61.0,65.000000
...,...,...,...,...,...,...,...,...,...,...
99995,20.000000,1.000000,3357.428033,279.370703,0.000000,23.459291,8.373307,7.678138,46.0,9.276035
99996,998.000000,4.000000,1.000000,6.000000,3.000000,23.459291,8.373307,7.678138,26.0,9.276035
99997,1799.000000,9.000000,81.000000,2.000000,0.000000,23.459291,8.373307,7.678138,60.0,5.000000
99998,2515.000000,4.000000,3357.428033,43.000000,106.000000,4.000000,8.373307,7.678138,16.0,5.000000


In [53]:
scaler = MinMaxScaler()

normalised_X2 = pd.DataFrame(scaler.fit_transform(Xx),columns =Xx.columns)

In [54]:
normalised_X2.head()

,REVENUE,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK
0,0.030460,0.144222,0.007079,0.011226,0.016664,0.007609,0.00613,0.00755,0.000000,0.021275
1,0.000050,0.000000,0.007079,0.011226,0.016664,0.007609,0.00613,0.00755,0.016393,0.021275
2,0.012687,0.222222,0.000873,0.011226,0.001229,0.000649,0.00613,0.00755,0.426230,0.041131
3,0.014365,0.144444,0.000700,0.000000,0.004039,0.001297,0.00613,0.00755,0.737705,0.005141
4,0.182180,0.511111,0.007079,0.005143,0.097454,0.090821,0.00613,0.00755,0.983607,0.164524


In [55]:
X = pd.concat([Xy,normalised_X2],axis = 1)

In [56]:
X.head()

,encoder__x0_DAKAR,encoder__x0_DIOURBEL,encoder__x0_FATICK,encoder__x0_KAFFRINE,encoder__x0_KAOLACK,encoder__x0_KEDOUGOU,encoder__x0_KOLDA,encoder__x0_LOUGA,encoder__x0_MATAM,encoder__x0_SAINT-LOUIS,...,REVENUE,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.030460,0.144222,0.007079,0.011226,0.016664,0.007609,0.00613,0.00755,0.000000,0.021275
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000050,0.000000,0.007079,0.011226,0.016664,0.007609,0.00613,0.00755,0.016393,0.021275
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012687,0.222222,0.000873,0.011226,0.001229,0.000649,0.00613,0.00755,0.426230,0.041131
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.014365,0.144444,0.000700,0.000000,0.004039,0.001297,0.00613,0.00755,0.737705,0.005141
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.182180,0.511111,0.007079,0.005143,0.097454,0.090821,0.00613,0.00755,0.983607,0.164524


In [57]:
test_predict = random_search.predict(X)
test_predict

array([1, 1, 0, ..., 0, 0, 0], dtype=int64)

In [58]:
test_predict_proba_ = random_search.predict_proba(X)

In [59]:
test_predict_proba_

array([[0.30479956, 0.69520044],
       [0.44948417, 0.55051583],
       [0.93066084, 0.06933919],
       ...,
       [0.98928964, 0.01071034],
       [0.9856196 , 0.01438038],
       [0.9381025 , 0.06189749]], dtype=float32)

In [62]:
new_test = pd.DataFrame(test_predict_proba_,test.user_id)

In [63]:
new_test

,0,1
user_id,,
af900d87e73b7ff6509d2203df4704a98aa5f2a6,0.304800,0.695200
5335efd940280b82143272275637d1e65d37eadb,0.449484,0.550516
a581f4fa08677c26f83f643248c667e241043086,0.930661,0.069339
64f67177d0775262b8087a9e2e3b8061b6324ae6,0.977138,0.022862
0d6009a4594c4be22449b8d9cc01a0bcea98faea,0.989008,0.010992
...,...,...
c6bcb3336795a18eb6c0bc7e19078a0704ef4d7e,0.994378,0.005622
a44b4e44dc70115ed5bf971ebb4193dd536e87f0,0.993335,0.006665
a2f84faffbc995bd0e2d726fa4ffdb93f11646ed,0.989290,0.010710


# I will still be doing more hyper parameter tuning 